# Gaia epoch photometry

Fun files in ESCV format. They've got loads of lists of values to encode lightcurves.

Getting them into hipscatted format is tricky.

Let's just take a single CSV and convert it to parquet, using only astropy utilities.

In [37]:
# Imports
import pyarrow.parquet as pq
from astropy.io import ascii
import pandas as pd
import dask.dataframe as dd
import dask

In [9]:
gaia_file = "/data3/epyc/data3/hipscat/raw/gaia/epoch_photometry/EpochPhotometry_786097-786431.csv.gz"
gaia_parqueted_file = '/data3/epyc/data3/hipscat/raw/gaia/epoch_parquet/epoch_data_786097-786431.parquet'



In [39]:
astropy_table = ascii.read(gaia_file, format='ecsv')
astropy_table.write(gaia_parqueted_file)

In [10]:
parquet_file = pq.ParquetFile(gaia_parqueted_file)
parquet_file.metadata

  created_by: parquet-cpp-arrow version 14.0.2
  num_columns: 48
  num_rows: 2088
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 37702

In [6]:
parquet_file.schema

required group field_id=-1 schema {
  optional int64 field_id=-1 solution_id;
  optional int64 field_id=-1 source_id;
  optional int32 field_id=-1 n_transits (Int(bitWidth=16, isSigned=true));
  optional group field_id=-1 transit_id (List) {
    repeated group field_id=-1 list {
      optional int64 field_id=-1 element;
    }
  }
  optional group field_id=-1 g_transit_time (List) {
    repeated group field_id=-1 list {
      optional double field_id=-1 element;
    }
  }
  optional group field_id=-1 g_transit_flux (List) {
    repeated group field_id=-1 list {
      optional double field_id=-1 element;
    }
  }
  optional group field_id=-1 g_transit_flux_error (List) {
    repeated group field_id=-1 list {
      optional double field_id=-1 element;
    }
  }
  optional group field_id=-1 g_transit_flux_over_error (List) {
    repeated group field_id=-1 list {
      optional float field_id=-1 element;
    }
  }
  optional group field_id=-1 g_transit_mag (List) {
    repeated group field

1. There's no ra/dec in there. Or really anything that tells us where in the sky this is (well, except for the source id, which **I'll get to**).

2. All of those repeated groups, like:

```
  optional group field_id=-1 transit_id (List) {
    repeated group field_id=-1 list {
      optional int64 field_id=-1 element;
    }
  }
```

That's not something that pandas handles very well.

What do I mean by not very well? Let's read the parquet with pandas, and write it back out. Are the lists preserved?

In [23]:
frame = pd.read_parquet(gaia_parqueted_file)

## Peek inside - does this look like a list?
## array([...]) - good enough for me!
frame.iloc[0]["transit_id"]

array([20017163183592008, 21758024825582601, 21762117108692400,
       22753539556143543, 22757631798867499, 22767372618699860,
       29947205190324581, 29956946004570871, 31863440876136253,
       31867533181108293, 32776061037065786, 32780153335243797,
       38199779899491908, 38203872185599193, 38642390290141290,
       38646482556732364, 38656223409890647, 40783479766397717,
       40787572058132961, 42873722790520017, 42883463683257274,
       42887555965702497, 42897296698663327, 43298421910037765,
       43312254938526537, 43316347240499082, 43330180524316492,
       48083812404442722, 48087904691868613, 48097645439378193,
       48637065320350794, 48641157607769040, 50737107953652419,
       52988788219208436, 52992880552501423, 53002621394236845,
       53006713664090825, 53338685724993318, 53348426448511468,
       53352518725189797, 53362259578343984, 53366351912956022,
       58799833949549537, 58803926219268662, 59726231636813538,
       61549809712780074, 61553902000845

In [40]:
frame = astropy_table.to_pandas()
frame.iloc[0]["transit_id"]

array([20017163183592008, 21758024825582601, 21762117108692400,
       22753539556143543, 22757631798867499, 22767372618699860,
       29947205190324581, 29956946004570871, 31863440876136253,
       31867533181108293, 32776061037065786, 32780153335243797,
       38199779899491908, 38203872185599193, 38642390290141290,
       38646482556732364, 38656223409890647, 40783479766397717,
       40787572058132961, 42873722790520017, 42883463683257274,
       42887555965702497, 42897296698663327, 43298421910037765,
       43312254938526537, 43316347240499082, 43330180524316492,
       48083812404442722, 48087904691868613, 48097645439378193,
       48637065320350794, 48641157607769040, 50737107953652419,
       52988788219208436, 52992880552501423, 53002621394236845,
       53006713664090825, 53338685724993318, 53348426448511468,
       53352518725189797, 53362259578343984, 53366351912956022,
       58799833949549537, 58803926219268662, 59726231636813538,
       61549809712780074, 61553902000845

In [24]:
frame.to_parquet("re-parqueted.parquet")

In [25]:
parquet_file = pq.ParquetFile("re-parqueted.parquet")
parquet_file.schema

required group field_id=-1 schema {
  optional int64 field_id=-1 solution_id;
  optional int64 field_id=-1 source_id;
  optional int32 field_id=-1 n_transits (Int(bitWidth=16, isSigned=true));
  optional group field_id=-1 transit_id (List) {
    repeated group field_id=-1 list {
      optional int64 field_id=-1 element;
    }
  }
  optional group field_id=-1 g_transit_time (List) {
    repeated group field_id=-1 list {
      optional double field_id=-1 element;
    }
  }
  optional group field_id=-1 g_transit_flux (List) {
    repeated group field_id=-1 list {
      optional double field_id=-1 element;
    }
  }
  optional group field_id=-1 g_transit_flux_error (List) {
    repeated group field_id=-1 list {
      optional double field_id=-1 element;
    }
  }
  optional group field_id=-1 g_transit_flux_over_error (List) {
    repeated group field_id=-1 list {
      optional float field_id=-1 element;
    }
  }
  optional group field_id=-1 g_transit_mag (List) {
    repeated group field

So far, so good. I look so melodramatic right now. 

Well, let's try introducing some more pandas/dask operations. I've got the `_hipscat_index` of the source IDs for JUST the 2088 rows in this shard. It was kind of annoying to generate, and really requires dask.

In [29]:
%%time
left_table = pd.read_parquet(
        path="single_partition_hipscat_index.parquet",
        engine="pyarrow",
    )
right_table = pd.read_parquet(
        path=gaia_parqueted_file,
        engine="pyarrow",
    )
result = left_table.merge(right_table, how="right", left_index=True, right_on="source_id")
result.iloc[0]["transit_id"]

array([20017163183592008, 21758024825582601, 21762117108692400,
       22753539556143543, 22757631798867499, 22767372618699860,
       29947205190324581, 29956946004570871, 31863440876136253,
       31867533181108293, 32776061037065786, 32780153335243797,
       38199779899491908, 38203872185599193, 38642390290141290,
       38646482556732364, 38656223409890647, 40783479766397717,
       40787572058132961, 42873722790520017, 42883463683257274,
       42887555965702497, 42897296698663327, 43298421910037765,
       43312254938526537, 43316347240499082, 43330180524316492,
       48083812404442722, 48087904691868613, 48097645439378193,
       48637065320350794, 48641157607769040, 50737107953652419,
       52988788219208436, 52992880552501423, 53002621394236845,
       53006713664090825, 53338685724993318, 53348426448511468,
       53352518725189797, 53362259578343984, 53366351912956022,
       58799833949549537, 58803926219268662, 59726231636813538,
       61549809712780074, 61553902000845

Raw pandas does ok. What about dask pandas?

In [38]:
%%time
dask.config.set({"dataframe.convert-string": False})
left_table = dd.read_parquet(
        path="/astro/users/mmd11/git/scripts/epyc/gaia/single_partition_hipscat_index.parquet",
        engine="pyarrow",
        filesystem="arrow",
    )
right_table = dd.read_parquet(
        path=gaia_parqueted_file,
        engine="pyarrow",
        filesystem="arrow",
    )
result = left_table.merge(right_table, how="right", left_index=True, right_on="source_id")
result = result.compute()
result.iloc[0]["transit_id"]

CPU times: user 251 ms, sys: 64.6 ms, total: 315 ms
Wall time: 306 ms


array([20017163183592008, 21758024825582601, 21762117108692400,
       22753539556143543, 22757631798867499, 22767372618699860,
       29947205190324581, 29956946004570871, 31863440876136253,
       31867533181108293, 32776061037065786, 32780153335243797,
       38199779899491908, 38203872185599193, 38642390290141290,
       38646482556732364, 38656223409890647, 40783479766397717,
       40787572058132961, 42873722790520017, 42883463683257274,
       42887555965702497, 42897296698663327, 43298421910037765,
       43312254938526537, 43316347240499082, 43330180524316492,
       48083812404442722, 48087904691868613, 48097645439378193,
       48637065320350794, 48641157607769040, 50737107953652419,
       52988788219208436, 52992880552501423, 53002621394236845,
       53006713664090825, 53338685724993318, 53348426448511468,
       53352518725189797, 53362259578343984, 53366351912956022,
       58799833949549537, 58803926219268662, 59726231636813538,
       61549809712780074, 61553902000845

Ah hah! There it is! The `transit_id` column is now a string representation of the real array data. 

And that's annoying. Because I don't want to use raw pandas to create the monster merge of 